# Yolo（you only look once）
<img src="https://pjreddie.com/media/image/sayit.jpg"/>

* 主页：https://pjreddie.com/darknet/yolo/
* 背景：
    - 一种单阶的目标检测器算法
    - 高效、快速
    - GitHub: https://github.com/pjreddie/darknet
        - 基于C和CUDA、CUDNN实现的框架，叫做darknet，用这个框架训练的Yolo模型
        - 从卷积、优化器、到模型、检测器训练全部都是C语言实现的，可以找到实现的每一个代码
    - 版本有Yolo、YoloV2、YoloV3、YoloV4、YoloV5
        - 其中Yolo、YoloV2、YoloV3，作者Joseph Redmon，也是Yolo的发起人，Darknet的创作者
            - 于2020年约2月，宣布退出计算机视觉：https://huanqiukexue.com/a/qianyan/xinxi__nenyuan/2020/0224/29238.html
        - 其中YoloV4出现在2020年4月，作者是AlexeyAB：https://github.com/AlexeyAB/darknet
            - 一个Yolo的拥护者，为Yolo提供了大量支持，Windows支持友好等工作
            - 受到作者点赞
            - 依旧基于C语言的darknet实现
            - V4的Pytorch实现：https://github.com/WongKinYiu/ScaledYOLOv4
        - 其中YoloV5出现在2020年5月，作者是ultralytics：https://github.com/ultralytics/yolov5
            - 一个马赛克增广的提出者
            - 基于Pytorch的实现版本
            - 官网集成：https://pytorch.org/hub/ultralytics_yolov5/

### YoloV3结构图：
<img src="yolov3.png"/>

### YoloV4结构图：
<img src="yolov4.png"/>

<img src="scaled-yolov4.png"/>

### YoloV5结构图：
<img src="yolov5.png"/>

# 概念1：Apex（Automatic Mixed Precision）
* 自动混合精度库，用于降低训练显存占用，提升训练速度
    - 如果没有FP16支持的显卡，这个选项没有意义
        - FP16是半精度的意思，以前的显卡仅仅支持FP32，对于FP16需要进行FP16到FP32转换，然后使用FP32运算，再转换回FP16。如果支持FP16，则是显卡硬件上支持FP16格式数据直接运算。由于计算精度底，相比FP32效率大大提升
        - 同样，由于是FP16，16个bit表示浮点数，造成可以表示的范围和精度有限，数值越大，精度越差。与ReLU6相呼应
* GitHub地址：https://github.com/NVIDIA/apex
    - 安装方法：
        ```bash
        git clone https://github.com/NVIDIA/apex
        cd apex
        pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
        ```
        - 如果安装出错，通常都是cuda环境上存在问题，例如：
            - CUDA_HOME=xxx，如果写成了CUDA_HOME=$CUDA_HOME:xxx就是有问题的，造成nvcc无法找到。因为需要进行CPP的编译操作
            - 找不到cublas_v2.h，也是由于cuda安装时少了东西造成的。重新安装
    - 使用方法：
        ```Python
        from apex import amp
        model, optimizer = amp.initialize(model, optimizer, opt_level='O1', verbosity=0)
        
        # 对于Loss的计算，需要进行梯度裁剪
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        ```

# 概念2：DP（DataParallel）/DDP（DistributedDataParallel）
* 多GPU同时训练时的并行方法
* DP已经淘汰，速度慢效果差，使用ParameterServer模式，显存占用多
* 欢迎来到DDP，基于Ring-AllReduce实现不同显卡的梯度Reduce操作（平均）
    - 使用方法：
        ```Python
        # 初始化后端
        import torch.distributed as dist
        dist.init_process_group(backend='nccl', init_method='env://')

        # 使用同步BN，不同卡之间传递均值方差
        model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model).to(device)
        
        # 使用DDP
        from torch.nn.parallel import DistributedDataParallel as DDP
        model = DDP(model, device_ids=[rank], output_device=rank)
        
        # 对于DataLoader的处理，需要使用DistributedSampler：
        train_sampler = torch.utils.data.distributed.DistributedSampler(dataset)
        dataloader = torch.utils.data.DataLoader(dataset,
                                                 batch_size=batch_size,
                                                 num_workers=nw,
                                                 sampler=train_sampler,
                                                 pin_memory=True,
                                                 collate_fn=LoadImagesAndLabels.collate_fn)
        
        #训练时，设置epoch：dataloader.sampler.set_epoch(epoch)
        #其余更正常训练一样
        ```
    - 带有DDP程序，启动时，第一种做法是基于python命令行：
        - 启动4个显卡执行训练，指定给main.py的参数是--program_args=value。master_port指定为6666，如果同时执行不同训练程序，需要指定不同master_port
        - CUDA_VISIBLE_DEVICES="0,1,2,3"是设置环境变量，让当前程序执行时，0，1，2，3显卡可见
        ```bash
        > CUDA_VISIBLE_DEVICES="0,1,2,3" python -m torch.distributed.launch --nproc_per_node 4 --master_port 6666 main.py --program_args=value
        ```
        - 对于main.py，需要接受一个参数--local_rank，该local_rank指定为序号，0、1、2、3。通过区分local_rank，实现不同逻辑，例如local_rank=0时打印loss和存储模型等等
    - 带有DDP程序，启动时，第二种做法是基于torch.multiprocessing.spawn方法实现
        - 该方法实现可以直接启动就是多卡

# trick
1. tqdm库，可以实现计算时的进度条并且告诉你时间

In [15]:
from tqdm import tqdm
import time

value_sum = 0
miter = range(1, 101)
bar = tqdm(miter, desc="For循环")
for i in bar:
    time.sleep(0.1)
    value_sum += i
    bar.set_description(f"For index = {i}")

print(f"value_sum = {value_sum}")

For index = 100: 100%|██████████| 100/100 [00:10<00:00,  9.75it/s]

value_sum = 5050


2. nms，可以使用torchvision.ops.nms函数实现GPU加速版本的nms计算，高效率